# Vulnerability Estimation

### read in census tract populations from exposure analysis

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
shapefile_folder = 'CensusTracts'
population_gdf_path = os.path.join(shapefile_folder, 'census_tracts_with_population.shp')
population_tracts_gdf = gpd.read_file(population_gdf_path)
population_tracts_gdf.head(25)


,index,Population,geometry
0,01063060102,2222.0,"POLYGON ((-724270.732 3709348.208, -724269.218..."
1,01063060101,1602.0,"POLYGON ((-721733.890 3701425.390, -721211.080..."
2,01069040802,3933.0,"POLYGON ((-498521.220 3494255.431, -498516.451..."
3,01069040204,6537.0,"POLYGON ((-498583.226 3504971.637, -498577.853..."
4,01069041902,4821.0,"POLYGON ((-484900.872 3491577.624, -484895.054..."
5,01069040205,5808.0,"POLYGON ((-502770.358 3503905.235, -502742.934..."
6,01069040203,2945.0,"POLYGON ((-502767.667 3504921.191, -502666.204..."
7,01069040801,5448.0,"POLYGON ((-500428.769 3500358.761, -500416.723..."
8,01069041901,3047.0,"POLYGON ((-488333.057 3493935.746, -488256.069..."
9,01069040206,4135.0,"POLYGON ((-501435.632 3501596.100, -501427.119..."


## Read in state boundaries and hurricane track data

In [3]:
# load us states
us_state_path = os.path.join('cb_2018_us_state_500k', 'cb_2018_us_state_500k.shp')
us_states = gpd.read_file(us_state_path)
us_states = us_states.to_crs(epsg = 32618)
# read all hurricane paths
hurricane_pathway = os.path.join('combined_tracks', 'combined_tracks_points.shp')
all_hurricane_points = gpd.read_file(hurricane_pathway) # read file
all_hurricane_points = all_hurricane_points.to_crs(epsg = 32618)


# initialize dataframe to store 
### hazard (windspeed)
### exposure (populations) and 
### vulnerability (damage) from each hurricane

In [18]:
hurricane_classification_df = pd.DataFrame()

# Read in Billion Dollar Disaster Damage Estimates


In [6]:
hurricane_losses = pd.read_csv('billion_dollar_losses.csv')

## Get list of all years with a hurricane

In [9]:
all_years = np.sort(all_hurricane_points['TCYR'].unique())
print(all_years)

[2010. 2011. 2012. 2013. 2014. 2015. 2016. 2017. 2018. 2019. 2020. 2021.
 2022. 2023. 2024.]


# Loop through each hurricane in each year to find damage matches
## Lookup all names of each hurricane, find name + correct year in damage database
### Match damage estimate with hurricane ID

In [19]:
for year_num in all_years:
  this_yr_hurricanes = all_hurricane_points[all_hurricane_points['TCYR'] == year_num]
  hurricane_nums = this_yr_hurricanes['STORMNUM'].unique()
  
  # set ids for individual hurricanes
  for strm_num in hurricane_nums:
    hurricane_id = str(int(year_num)) + '_' + str(int(strm_num))
    hurricane_classification_df.loc[hurricane_id, 'Total Damage']  = 0.0

    # slice dataframe to find only current hurricane
    this_hurricane = this_yr_hurricanes[this_yr_hurricanes['STORMNUM'] == strm_num]
    # get list of all names the hurricane was called
    hurricane_names = this_hurricane['STORMNAME'].unique()
    # get year to match with hurricane name
    hurricane_year = int(this_hurricane.loc[this_hurricane.index[-1], 'TCYR'])

    # loop through all potential names to match with damage list
    for hurricane_name in hurricane_names:
      if hurricane_name:
        loss_check = hurricane_losses[hurricane_losses['Hurricane'].str.contains(hurricane_name, na=False, case=False)]
        if len(loss_check.index) > 0:
          # name match must occur in correct year
          if int(loss_check.loc[loss_check.index[0], 'Year']) == hurricane_year:
            hurricane_classification_df.loc[hurricane_id, 'Total Damage'] = loss_check.loc[loss_check.index[0], 'Cost'] * 1.0
            break
print(hurricane_classification_df[hurricane_classification_df['Total Damage'] > 0.0])


         Total Damage
2011_9           18.8
2011_13           3.5
2012_18          88.5
2016_14          13.1
2017_9          160.0
2017_11          64.0
2017_15         115.2
2018_6           30.0
2018_14          31.2
2019_5            2.0
2019_11           6.2
2020_8            1.3
2020_9            5.8
2020_13          28.1
2020_19           8.8
2020_26           3.5
2020_28           5.3
2020_29           1.8
2021_5            1.4
2021_6            1.5
2021_14           1.2
2022_7            2.7
2022_9          119.6
2022_17           1.1
2024_2            7.2
2024_4            2.5
2024_6            1.3
2024_9           78.7
2024_14          34.3


# Find exposed populations for billion-dollar disasters storms
## first set buffer diameters

In [20]:
buffer_radii = [200000.0, 150000.0, 100000.0, 50000.0]

### only do exposure calculation on storms with damage

In [25]:
billion_dollar_hurricanes = hurricane_classification_df[hurricane_classification_df['Total Damage'] > 0.0]
for storm_index in billion_dollar_hurricanes.index:
  print(storm_index)
  hurricane_year = int(storm_index.split('_')[0])
  hurricane_num = int(storm_index.split('_')[1])
  this_yr_hurricanes = all_hurricane_points[all_hurricane_points['TCYR'] == hurricane_year]
  this_hurricane = this_yr_hurricanes[this_yr_hurricanes['STORMNUM'] == hurricane_num]

  # initialize hazard and exposure values for this storm
  hurricane_classification_df.loc[storm_index, 'Landfall Windspeed'] = 0.0
  for buffer_radius in buffer_radii:
    hurricane_classification_df.loc[storm_index, 'Exposed Population < ' + str(int(buffer_radius/1000.0)) + 'km'] = 0.0

  # calculate exposure to hazards > than 75mph
  for index, row in this_hurricane.iterrows():
    if row['INTENSITY'] > 75.0 / 1.15:    
      for buffer_radius in buffer_radii:
        buffer_geom = row['geometry'].buffer(buffer_radius)
        # create new dataframe for circle created by radius
        gdf_point_buffer = gpd.GeoDataFrame([0,], crs = this_hurricane.crs, geometry = [buffer_geom,])      
        # find intersection of hurricane radius with census tracts
        exposed_tracts = gpd.sjoin(population_tracts_gdf, gdf_point_buffer, how = 'inner', predicate = 'intersects')
        if len(exposed_tracts) > 0:
          hurricane_classification_df.loc[storm_index, 'Landfall Windspeed'] = max(row['INTENSITY'] * 1.15, hurricane_classification_df.loc[storm_index, 'Landfall Windspeed'])
          # find total population contained in those census tracts
          total_population = 0 
          for indexp, rowp in exposed_tracts.iterrows():
            total_population += np.sum(rowp['Population'])
          hurricane_classification_df.loc[storm_index, 'Exposed Population < ' + str(int(buffer_radius/1000.0)) + 'km'] += total_population
hurricane_classification_df.to_csv('hurricane_hazard_exposure_vulnerability.csv')

2011_9
2011_13
2012_18
2016_14
2017_9
2017_11
2017_15
2018_6
2018_14
2019_5
2019_11
2020_8
2020_9
2020_13
2020_19
2020_26
2020_28
2020_29
2021_5
2021_6
2021_14
2022_7
2022_9
2022_17
2024_2
2024_4
2024_6
2024_9
2024_14
